# ANISE

ANISE is a modern rewrite of NAIF SPICE, written in Rust and providing interfaces to other languages including Python.

Evidently, this tutorial applies to the Python usage of ANISE.

## Goal
By the end of this tutorial, you should know how to build a data frame containing the Sun probe Earth angle of a given spacecraft BSP and plot that information. Your exercise will be to confirm that this calculation is correct by computing the Sun elevation at nadir below the spacecraft as detailed in tutorial 04.## Loading the latest orientation and planetary data

Let's start by installing ANISE: `pip install anise`

## Load a BSP containing a spacecraft trajectory

In this tutorial, we will use the `gmat-hermite.bsp` file which contains some trajectory data build in GMAT and used in ANISE for validation purposes. Although the Sun probe Earth angle _typically_ applies to spacecraft trajectories, the calculation works for any two objects.

In [2]:
from anise import MetaAlmanac
almanac = MetaAlmanac.latest().load("../../data/gmat-hermite.bsp")
almanac.describe(spk=True)

=== SPK #0 ===
┌─────────────┬──────────────────────┬─────────────┬───────────────────────────────────┬───────────────────────────────────┬────────────┬──────────────────────┐
│ Name        │ Target               │ Center      │ Start epoch                       │ End epoch                         │ Duration   │ Interpolation kind   │
├─────────────┼──────────────────────┼─────────────┼───────────────────────────────────┼───────────────────────────────────┼────────────┼──────────────────────┤
│ SPK_SEGMENT │ body -10000001 J2000 │ Earth J2000 │ 2000-01-01T12:00:32.183927328 TDB │ 2000-01-01T15:20:32.183931556 TDB │ 3 h 20 min │ Hermite Unequal Step │
└─────────────┴──────────────────────┴─────────────┴───────────────────────────────────┴───────────────────────────────────┴────────────┴──────────────────────┘
=== SPK #1 ===
┌────────────────┬─────────────────────────────┬─────────��─────────────────────┬───────────────────────────────────┬───────────────────────────────────┬────────────

We have successfully loaded two BSP files, one with the planetary data and the other with spacecraft data. We now know that this spacecraft has the ID `-10000001`. We can actually query the Almanac for the precise start and stop epochs (returned as hifitime `Epoch` objects) of this ID in our loaded files.

In [3]:
start_epoch, stop_epoch = almanac.spk_domain(-10000001)
print(start_epoch, stop_epoch)

2000-01-01T11:59:28.000000021 UTC 2000-01-01T15:19:28.000000226 UTC


The Sun Probe Earth angle is the angle between a probe and the Sun and the probe the Earth. It allows one to know whether the point exactly nadir (i.e. below) the spacecraft is illuminated by the Sun or not. This is helpful information if the spacecraft carries a visible light camera and needs to take pictures when it's daytime below the spacecraft.

Let's look at the signature of this function.

In [7]:
almanac.sun_angle_deg?

Signature: almanac.sun_angle_deg(target_id, observer_id, epoch)
Docstring:
Returns the angle (between 0 and 180 degrees) between the observer and the Sun, and the observer and the target body ID.
This computes the Sun Probe Earth angle (SPE) if the probe is in a loaded, its ID is the "observer_id", and the target is set to its central body.

# Geometry
If the SPE is greater than 90 degrees, then the celestial object below the probe is in sunlight.

## Sunrise at nadir
```text
Sun
 |  \      
 |   \
 |    \
 Obs. -- Target
```
## Sun high at nadir
```text
Sun
 \        
  \  __ θ > 90
   \     \
    Obs. ---------- Target
```

## Sunset at nadir
```text
         Sun
       /  
      /  __ θ < 90
     /    /
 Obs. -- Target
```

# Algorithm
1. Compute the position of the Sun as seen from the observer
2. Compute the position of the target as seen from the observer
3. Return the arccosine of the dot product of the norms of these vectors.
Type:      builtin_function_or_method

One will note that this function is generic to what the "probe" SPK ID is ("observer") and what its central object should be instead of Earth ("target").

The other crucial point here is that this is one of the few functions where the object _ID_ is required instead of a frame. This is because the Almanac will compute everything in the J2000 frame. Don't worry, if you have frame objects instead, you may use the `sun_angle_deg_from_frame` function instead.

Let's see what is the SPE of our spacecraft at the start of the trajectory.


In [6]:
from anise.astro.constants import CelestialObjects
almanac.sun_angle_deg(-10000001, CelestialObjects.EARTH, start_epoch)

83.87312777296376

A angle of less than 90 degrees means that the nadir point is in the darkness. Let's look at the evolution of the SPE over the duration of the trajectory.

## Package installation for plotting

In [12]:
%pip install "polars[plot]" hvplot geoviews # geoviews for geographic data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.2/511.2 kB 690.4 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 14.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.5 MB/s eta 0:00:0031m72.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 13.8 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 47.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 49.5 MB/s eta 0:00:0031m56.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 42.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kern

## Evolution of SPE over time

We'll plot the change in Sun probe Earth angle over time. We'll also grab the latitude and longitude data so we can plot the position of the spacecraft above the Earth at those times (as a separate plot).

In [38]:
from anise.time import TimeSeries, Unit, Epoch
from anise.astro import Frame
from anise.astro.constants import Frames, Orientations

import polars as pl
from datetime import datetime

def hifitime_to_datetime(e: Epoch) -> datetime:
    return datetime.fromisoformat(str(e).replace(" UTC", "")[:23])

epochs = []
spe_deg = []
lat_deg = []
long_deg = []

# Let's be sure to load the PCK data, which includes the frame information such as the shape of
# the ellipsoid and how to compute the rotation of the body fixed frames
almanac = almanac.load("../../data/pck08.pca")

SC_ID = -10000001
SC_J2K = Frame(SC_ID, Orientations.J2000)

for epoch in TimeSeries(start_epoch, stop_epoch, Unit.Minute*1, inclusive=True):
    epochs += [hifitime_to_datetime(epoch)]
    spe_deg += [almanac.sun_angle_deg(CelestialObjects.EARTH, SC_ID, epoch)]
    # Grab position of the spacecraft in the IAU Earth frame
    sc_iau_earth = almanac.transform(SC_J2K, Frames.IAU_EARTH_FRAME, epoch)
    lat_deg += [sc_iau_earth.latitude_deg()]
    long_deg += [sc_iau_earth.longitude_deg()]

In [39]:
# Build the data frame
df = pl.DataFrame(
    {
        'Epoch': epochs,
        'Sun Probe Earth angle (deg)': spe_deg,
        'Latitude (deg)': lat_deg,
        'Longitude (deg)': long_deg
    }
)

In [43]:
import hvplot.polars
from bokeh.models.formatters import DatetimeTickFormatter

formatter = DatetimeTickFormatter(days='%d/%m') # Make the world a better place

df.hvplot(x="Epoch", y="Sun Probe Earth angle (deg)", xformatter=formatter, 
          title="Sun probe Earth angle over time", hover_cols=["Latitude (deg)", "Longitude (deg)"])

:Curve   [Epoch]   (Sun Probe Earth angle (deg),Latitude (deg),Longitude (deg))

In [41]:
df.hvplot.points('Longitude (deg)', 'Latitude (deg)', geo=True, color='red', tiles='ESRI', xlim=(0, 360), ylim=(-60, 60), hover_cols=["Epoch", "Sun Probe Earth angle (deg)"])

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude (deg),Latitude (deg)]   (Epoch,Sun Probe Earth angle (deg))

## Exercise

The goal of this exercise is to show that the SPE is essentially the Sun elevation angle from the position exactly nadir of the vehicle plus 90 degrees (or so, since the Earth isn't a perfect sphere). It also shows you how to combine the different functionality you've seen with the other tutorials to solve similar problem.

_Note:_ this is the `verify_geometry` Rust test, but rebuilt in Python.

1. For the whole spacecraft trajectory, build the locality exactly nadir of it at each epoch (remember that a `TimeSeries` instance can only be used once, so you'll need to rebuild a new one). For this step, initialize a new `Orbit` instance from the latitude and longitude constructor using the position of the spacecraft in the IAU Earth frame as we did above.
2. At each epoch, grab the state of the Sun as seen from the Earth (both can be in the J2000 frame since the AER computation will transform the states into the correct frames anyway).
3. Call the azimuth, elevtion, and range function of your loaded Almanac, and store the elevation of the Sun as seen from the point exactly nadir of the spacecraft.
4. Plot the elevation data in degrees compared to the SPE angle calculated above.